In [1]:
import pykerrp2p as rt

from mpmath import mp
# 113 for quad precision, 237 for oct precision
mp.prec = 237
print(mp.dps)

import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "axes.prop_cycle": plt.cycler('color', ['k', 'r', 'b', 'g'])})

70


In [2]:
def rc_range(a):
    return (
        4 * mp.power(mp.cos(mp.acos((-a).value)/3), 2) + mp.mpf("1e-10"), 
        4 * mp.power(mp.cos(mp.acos(a.value)/3), 2) - mp.mpf("1e-10")
    )

def sign_to_num(x):
    if x == rt.Sign.POSITIVE: return +1
    elif x == rt.Sign.NEGATIVE: return -1
    else: return np.nan
        
def conserve_to_ob(a, lam, eta, theta_o, nu_theta_o): ## nu_theta_o takes number +1 or -1
    alpha = -lam / mp.sin(theta_o.value)
    beta = nu_theta_o * mp.sqrt((eta + mp.power(a.value,2) * mp.power(mp.cos(theta_o.value),2) - mp.power(lam.value,2) / mp.power(mp.tan(theta_o.value),2)).value)
    return alpha, beta

# Basic Usages

In [3]:
"""set parameters"""
print("Notice that we fixed phi_s = 0 in this code for simplicity.")
params = rt.ForwardRayTracingParamsFloat256()
params.a.value = "0.8"
params.r_s.value = "10"
params.theta_s.value = mp.nstr(mp.mpf("89.9") * mp.pi / mp.mpf("180"), mp.dps)
params.nu_r = rt.Sign.NEGATIVE
params.nu_theta = rt.Sign.NEGATIVE
params.r_o.value = "1000"

Notice that we fixed phi_s = 0 in this code for simplicity.


In [4]:
"""calculate the final direction of a ray given source position and (rc,d)"""
params.rc.value = '3.2'
params.log_abs_d.value = '-0.5' ## 'log' takes 10 as the base
params.d_sign = rt.Sign.POSITIVE
params.calc_t_f = True
params.rc_d_to_lambda_q()
res = rt.calc_ray_Float256(params)

print("%11s" % "lam: ", params.lam)
print("%11s" % "eta: ", params.q*params.q)
print("%11s" % "q: ", params.q, "\n")
print("%11s" % "state: ", res.ray_status)
print("%11s" % "theta_f: ", res.theta_f)
print("%11s" % "phi_f: ", res.phi_f)
print("%11s" % "t_f: ", res.t_f)
print("%11s" % "n: ", res.n_half)

      lam:  -2.761619769027745661530993527119575813019392684677938312749743077808627572
      eta:  28.94863977029793708640849394628577214858643487686645577302665589675438619
        q:  5.380394016268505319660997745745719523712113041442710108352492691663132135 

    state:  RayStatus.NORMAL
  theta_f:  2.540356981852811433239965351624448074503589797116938207125546754473732558
    phi_f:  -4.977691330899621738118787843880283016294704894811339441673536443371582238
      t_f:  1038.026055852392155889009521913993125083448695499504030846463625492776947
        n:  1.622843009206923658546106643472402987961343388443649569242595191788983888


In [5]:
"""impact parameters of the image"""
nu_theta_o = sign_to_num(params.nu_theta) * (-1)**res.m
al, be = conserve_to_ob(params.a, params.lam, params.q*params.q, res.theta_f, nu_theta_o)
print(nu_theta_o)
print("alpha: ", al)
print("beta: ", be)

-1
alpha:  4.882103132446983272544693227635212842362753962143566301401060960459149077
beta:  -3.629802686203412622404963385292413100659349112952641096544207197081656


In [6]:
"""find_root_Float256: treat phi and phi + 2pi as equivalent"""
theta_target = 2.5
phi_target_normalized = -5.0 % (2*np.pi)
phi_target_normalized

1.2831853071795862

In [7]:
tol = rt.Float256("1e-60") ## residue tolerance 
res2 = rt.find_root_Float256(params, rt.Float256(mp.nstr(theta_target, mp.dps)), rt.Float256(mp.nstr(phi_target_normalized, mp.dps)), tol)

print("success = ", res2.success,"\n")
if res2.success:
    root = res2.root
    print("%11s" % "rc: ", root.rc)
    print("%11s" % "log_abs_d: ", root.log_abs_d)
    print("%11s" % "lam: ", root.lam)
    print("%11s" % "eta: ", root.eta)
    print("%11s" % "q: ", mp.sqrt(mp.nstr(root.eta, mp.dps)),"\n")
    
    print("%11s" % "state: ", root.ray_status)
    print("%11s" % "theta_f: ", root.theta_f)
    print("%11s" % "phi_f: ", root.phi_f)
    print("%11s" % "t_f: ", root.t_f)
    print("%11s" % "n: ", root.n_half)
else:
    print("fail reason: ", res2.fail_reason)

success =  True 

       rc:  3.223886426223163269959399709991144305302717373920741116041822934146217398
log_abs_d:  -0.5195408229505847985119904090573119865910841801314606806701126931863143428
      lam:  -2.903346700089067288865175659152336288824652674863372687641640772619804679
      eta:  28.42412350227013048410573013654112176069004147118303146737257975472478821
        q:  5.331427904630253038090085297767850979410644649064304463254788441865 

    state:  RayStatus.NORMAL
  theta_f:  2.499999999999999999999999999999999999999999999999999999999999999999999801
    phi_f:  -5.000000000000000276925286766559005768394338798750211641949889184615635615
      t_f:  1038.237286327342515026380075321202215656745924216673923924556344906537321
        n:  1.634704676809194489597927040640992587358328713562945690211353346076707439


In [8]:
"""find_root_period_Float256: specify a period number of phi"""
theta_target = 2.5
phi_target_normalized = -5.0 % (2*np.pi)
period = int(np.floor(-5.0 / (2 * np.pi)))
(phi_target_normalized, period)

(1.2831853071795862, -1)

In [9]:
tol = rt.Float256("1e-60") ## residue tolerance 
res3 = rt.find_root_period_Float256(params, period, rt.Float256(mp.nstr(theta_target, mp.dps)), rt.Float256(mp.nstr(phi_target_normalized, mp.dps)), tol)

print("success = ", res3.success,"\n")
if res3.success:
    root = res3.root
    print("%11s" % "rc: ", root.rc)
    print("%11s" % "log_abs_d: ", root.log_abs_d)
    print("%11s" % "lam: ", root.lam)
    print("%11s" % "eta: ", root.eta)
    print("%11s" % "q: ", mp.sqrt(mp.nstr(root.eta, mp.dps)),"\n")
    
    print("%11s" % "state: ", root.ray_status)
    print("%11s" % "theta_f: ", root.theta_f)
    print("%11s" % "phi_f: ", root.phi_f)
    print("%11s" % "t_f: ", root.t_f)
    print("%11s" % "n: ", root.n_half)
else:
    print("fail reason: ", res3.fail_reason)

success =  True 

       rc:  3.223886426223163269959399709991144305302717373920741116041822934146217978
log_abs_d:  -0.5195408229505847985119904090573119865910841801314606806701126931863140394
      lam:  -2.903346700089067288865175659152336288824652674863372687641640772619808319
      eta:  28.42412350227013048410573013654112176069004147118303146737257975472478009
        q:  5.331427904630253038090085297767850979410644649064304463254788441864999 

    state:  RayStatus.NORMAL
  theta_f:  2.499999999999999999999999999999999999999999999999999999999999999999999819
    phi_f:  -5.000000000000000276925286766559005768394338798750211641949889184615632934
      t_f:  1038.23728632734251502638007532120221565674592421667392392455634490653733
        n:  1.634704676809194489597927040640992587358328713562945690211353346076707185
